In [ ]:
%sh
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/16.04/prod.list > /etc/apt/sources.list.d/mssql-release.list
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql17

In [ ]:
import os
import json

from concurrent import futures
import time

from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils
from pyspark.sql import SQLContext

from edm.batch_ingestion.raw_to_standardised.raw_to_standardized_processing import RawToStandardizeProcessor

In [ ]:
spark = SparkSession.builder.appName('').getOrCreate()
dbutils = DBUtils(spark)
sqlContext = SQLContext(spark)

In [ ]:
dbutils.widgets.text('raw_file_metadata', '')
dbutils.widgets.text('source_id', '')
dbutils.widgets.text('spn_id', '')
dbutils.widgets.text('tenant_id', '')
dbutils.widgets.text('kv_name', '')
dbutils.widgets.text('adls_account_name', '')
dbutils.widgets.text('config_path', '')

In [ ]:
raw_file_metadata = json.loads(dbutils.widgets.get('raw_file_metadata'))
source_id = dbutils.widgets.get('source_id')
spn_id = dbutils.widgets.get('spn_id')
tenant_id = dbutils.widgets.get('tenant_id')
kv_name = dbutils.widgets.get('kv_name')
adls_account_name = dbutils.widgets.get('adls_account_name')
config_path = dbutils.widgets.get('config_path')

In [ ]:
spn_password = os.getenv('SPN_PASSWORD', None)

In [ ]:
spn_credentials = dict()
spn_credentials['spn_id'] = spn_id
spn_credentials['spn_password'] = spn_password
spn_credentials['tenant_id'] = tenant_id

In [ ]:
def rts_runprocess(item):
  print(item)
  # Param Config Parser
  rts_obj = RawToStandardizeProcessor(
      source_id, item['SourceObjectId'], item['FileName'],
      spark, dbutils, spn_credentials=spn_credentials,
      kv_name=kv_name, adls_account_name=adls_account_name,
      config_path=config_path
  )
  rts_obj.run()

In [ ]:
# Leverage distributive processing
t1 = time.perf_counter()
with futures.ThreadPoolExecutor() as executor:
  result = executor.map(rts_runprocess,raw_file_metadata)
  print(result)

t2 = time.perf_counter()
t = t2 - t1
print(f"total {t} secs taken")